In [212]:
# An example of building up the structure of a figure as a Python dictionary
# and then constructing a graph object figure from that dictionary.

In [213]:
# https://plotly.com/python/animations/#using-a-slider-and-buttons
# https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv

In [214]:
from datetime import datetime

import pandas as pd
import plotly.graph_objects as go

In [215]:
import plotly.io as pio
pio.renderers.default = 'chrome' # https://community.plotly.com/t/plotly-doesnt-load-most-of-the-time/32095/3

In [216]:
dataset = pd.read_csv('output_data/time_series/rolling-time-series.csv')

In [217]:
dataset['Date_Confirmed'] = dataset['Date_Confirmed'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

In [218]:
import janitor
dataset = dataset.filter_date('Date_Confirmed','2020-03-01','2020-04-23')

In [219]:
# Make list of regions

regions = []

for region in dataset['Region']:
    if region not in regions:
        regions.append(region)

In [220]:
years = sorted(list(set(dataset['Date_Confirmed'])))

In [221]:
filled_df = []

for region in regions:
    for year in years:
        dataset_by_year = dataset[dataset['Date_Confirmed'] == year]
        dataset_by_year_and_region = dataset_by_year[dataset_by_year['Region'] == region]
        if len(dataset_by_year_and_region) == 0:
            dataset_by_year_less = dataset[dataset['Date_Confirmed'] < year]
            dataset_by_year_and_region_less = dataset_by_year_less[dataset_by_year_less['Region'] == region]
            if len(dataset_by_year_and_region_less) == 0:
                filled_df.append({'Date_Confirmed': year, 'Region': region, 'total':0, 'daily':0})
            else:
                last_dict = dataset_by_year_and_region_less.iloc[0].to_dict()
                last_dict['Date_Confirmed'] = year
                filled_df.append(last_dict)
        else:
            filled_df.append(dataset_by_year_and_region.iloc[0].to_dict())

In [222]:
dataset = pd.DataFrame(filled_df)

In [223]:
dataset['total'].max()

756407.0

In [224]:
dataset['daily'].max()

31595.0

In [225]:
dataset.tail()

,Date_Confirmed,Region,total,daily
313,2020-04-18,US,639921.0,29400.0
314,2020-04-19,US,668992.0,29071.0
315,2020-04-20,US,698093.0,29101.0
316,2020-04-21,US,727264.0,29171.0
317,2020-04-22,US,756407.0,29143.0


In [226]:
len(str(dataset['total'].max()))+1

9

In [227]:
# Make figure

fig_dict = {
    'data': [],
    'layout': {},
    'frames': []
}

In [228]:
# Fill in most of layout

fig_dict['layout']['xaxis'] = {'title': 'Casos Totales', 'type': 'log', 'range': [0,6]}
fig_dict['layout']['yaxis'] = {'title': 'Casos Nuevos', 'type': 'log', 'range': [0,6]}
fig_dict['layout']['hovermode'] = 'closest'
fig_dict['layout']['sliders'] = {
    'args': [
        'transition', {
            'duration': 400,
            'easing': 'cubic-in-out'
        }
    ],
    'initialValue': datetime.strptime('15-02-2020', '%d-%m-%Y').isoformat().split('T')[0],
    'plotlycommand': 'animate',
    'values': years,
    'visible': True
    }
fig_dict['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                                'fromcurrent': True, 'transition': {'duration': 300,
                                                                    'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False},
                                  'mode': 'immediate',
                                  'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]

sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Fecha:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}

In [229]:
# Initiate dicts

year = datetime.strptime('2020-03-05', '%Y-%m-%d')

for region in regions:
    
    dataset_by_year = dataset[dataset['Date_Confirmed'] == year]

    dataset_by_year_and_region = dataset_by_year[dataset_by_year['Region'] == region]
    
    data_dict = {
        'x': list(dataset_by_year_and_region['total']),
        'y': list(dataset_by_year_and_region['daily']),
        'mode': 'lines',
        'text': list(dataset_by_year_and_region['Region']),
    }
    
    data_dict_2 = {
        'x': list(dataset_by_year_and_region['total']),
        'y': list(dataset_by_year_and_region['daily']),
        'mode': 'lines',
        'text': list(dataset_by_year_and_region['Region']),
    }
    
    fig_dict['data'].append(data_dict)
    fig_dict['data'].append(data_dict_2)

In [230]:
# Make frames

for y, year in enumerate(years):
    
    frame = {'data': [], 'name': str(year.isoformat().split('T')[0])}
    frame_2 = frame.copy()
    
    dataset_by_year = dataset[dataset['Date_Confirmed'] <= year]
    
    for region in regions:
        dataset_by_year_and_region = dataset_by_year[
            dataset_by_year['Region'] == region]
            
        data_dict = {
                    'x': list(dataset_by_year_and_region['total']),
                    'y': list(dataset_by_year_and_region['daily']),
                    'mode': 'lines',
                    'text': list(dataset_by_year_and_region['Region']),
                    'textposition': 'bottom center',
                    'name': region,
            }
        
        data_dict_2 = {
                    'x': [list(dataset_by_year_and_region['total'])[-1]],
                    'y': [list(dataset_by_year_and_region['daily'])[-1]],
                    'mode': 'markers+text',
                    'text': [list(dataset_by_year_and_region['Region'])[-1]],
                    'textposition': 'bottom center',
                    'name': region,
            }
        
        frame['data'].append(data_dict)
        frame['data'].append(data_dict_2)
        
    fig_dict['frames'].append(frame)
    
    slider_step = {'args': [
        [year.isoformat().split('T')[0]],
        {'frame': {'duration': 300, 'redraw': True},
         'mode': 'immediate',
         'transition': {'duration': 300}}
    ],
        'label': year.isoformat().split('T')[0],
        'method': 'animate'}
    
    sliders_dict['steps'].append(slider_step)
    
fig_dict['layout']['sliders'] = [sliders_dict]

In [ ]:
fig = go.Figure(fig_dict)

fig.show()

In [206]:
#fig_dict['frames'][-1]